<a href="https://colab.research.google.com/github/vonnan/IBM-capstone-project/blob/main/IBM_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###IBM Capstone Project

In [6]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Webscraping from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

To create the above dataframe:

1. The dataframe will consist of three columns: PostalCode, 
Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.





### Use pd.read_html for webscraping the Toronto data

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood



In [17]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df1, df2, df3 = pd.read_html(url)

df1.columns = ["PostalCode", "Borough", "Neighborhood"]




### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [24]:
toronto = df1.loc[df1["Borough"] != "Not assigned"].copy() # remove the unassigned borough

toronto.sort_values("PostalCode", inplace = True)

###If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making. 

###In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.



In [25]:
toronto[toronto["Neighborhood"] == "Not assigned"]["Neighborhood"] = toronto[toronto["Neighborhood"] == "Not assigned"]["Borough"] #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

print("There are {} rows, i.e., postal codes in the dataframe".format(toronto.shape[0])) 

toronto.head()

There are 103 rows, i.e., postal codes in the dataframe


,PostalCode,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


### The above is for the first link submission, requirement 1-4

(Submit a link to your Notebook on your Github repository. (10 marks))

###we need to get the latitude and the longitude coordinates of each neighborhood.
First try use geocoder.google to get the latitude and longitude information

Step 1: Install geocoder and import it

In [ ]:
!pip install geocoder
import geocoder


In [ ]:
"""

def get_lat_long(postalcode):
  counter = 0
  location = None
  while (location == None):
    location = geocoder.google("{}, Toronto, Ontario".format(postalcode)).latlon
    counter += 1
    if counter ==10:
      break
  
  return location

toronto["location"] = toronto["PostalCode"].apply(get_lat_long)

toronto.head()
"""

###Geocoder.google is not reliable.
We have to user the webscraping to obtain the latitude/longitude for the Toronto borough/neighborhood

In [ ]:
url = "https://cocl.us/Geospatial_data"
PC_location = pd.read_csv(url)
PC_location.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
toronto = toronto.merge(PC_location, how = "left", on= "PostalCode")


In [27]:
print(toronto.shape[0])
toronto.head(12)

103


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Above is the second submission for the dataframe created with the latitude/longitude 

Below we are going to do the last part of the 3rd week assignment

In [28]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library




### We will focus on the 4 boroughs that contain "Toronto" in the Borough name; create a new dataframe of Toronto_city

In [35]:
toronto_city = toronto[toronto["Borough"].str.contains("Toronto")].copy().reset_index(drop=True)
toronto_city.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [38]:
boroughs = list(toronto_city["Borough"].unique())
colors =["red", "blue", "green", "purple"]


###Draw the map of Toronto

Step 1: get the latitude/longitude of Toronto

In [46]:
address = "Toronto, Ontario"

toronto_map = folium.Map

location = Nominatim(user_agent="toronto_explorer").geocode(address)

lat, lng = location.latitude, location.longitude
lat,lng

(43.6534817, -79.3839347)

Step 2: Draw the map of Toronto using Folium.Map

In [61]:
toronto_map = folium.Map([lat, lng], zoom_start=12)

for lat, lng, brgh, hood in zip(toronto_city["Latitude"], toronto_city["Longitude"], toronto_city["Borough"], toronto_city["Neighborhood"]):
  label = brgh + ": " + hood
  for i in range(4):
    if brgh == boroughs[i]:
      color = colors[i]
      
  folium.CircleMarker(
      [lat, lng],
      radius = 10,
      popup = label,
      fill = True,
      color = color,
      fill_color = color,
      fill_opacity = 0.3
  ).add_to(toronto_map)

In [52]:
toronto_map

###First, just use Kmeans clustering to divide the clustering based on latitude and longitude 

In [57]:
KM = KMeans(n_clusters= len(boroughs), n_init= 12, random_state=0)
X = toronto_city[["Latitude", "Longitude"]]
KM.fit(X)
colors = []
KM.labels_
for label in KM.labels_:
  if label == 1:
    color = "red"
  elif label == 2:
    color = "blue"
  elif label == 3:
    color = "green"
  else:
    color = "purple"
  colors.append(color)

for lat, lng, brgh, hood, color in zip(toronto_city["Latitude"], toronto_city["Longitude"], toronto_city["Borough"], toronto_city["Neighborhood"], colors):
  label = brgh + ": " + hood
  folium.CircleMarker(
      [lat, lng],
      radius = 4,
      popup = label,
      fill = True,
      color = color,
      fill_color = color,
      fill_opacity = 0.3
  ).add_to(toronto_map)

### KMeans is actually doing a good job. The prediction is correct 37 out of 39 PostalCodes. 

### Two wrong: Downtown Toronto: Christie is clustered to the rest of West Toronto; Central Toronto: The Annex, North Midtown, Yorkville is clustered to Downtown Toronto

In [60]:
toronto_map

In [ ]:
manhattan_onehot = pd.get_dummies(manhattan_venues[["Venue Category"]], prefix = "", prefix_sep = "")
manhattan_onehot["Neighborhood"] = manhattan_venues["Neighborhood"]
manhattan_onehot.head()
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()